In [236]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import joblib
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix

In [237]:
# Caricamento del dataset
file_path = 'data-istruzioni/Data.xlsx'
data = pd.read_excel(file_path)

# Rimozione colonne inutili
data_cleaned = data.drop(columns=['ID'])

In [198]:
#display(data_cleaned.head())
#display(data_cleaned.describe())

In [199]:
#data_cleaned.info()

In [238]:
X = data_cleaned.drop('Recidiva/Non_Recidiva', axis = 1)
y = data_cleaned['Recidiva/Non_Recidiva']

In [ ]:
#DECISION TREE

# Divisione train/test con proporzione 70%-30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Gestione valori mancanti
numerical_imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(numerical_imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(numerical_imputer.transform(X_test), columns=X_test.columns)

# Inizializzazione della matrice di confusione cumulativa
CM = np.zeros([len(y.unique()), len(y.unique())])
y_test_all = []
predictions_all = []

# Modello
dtree = tree.DecisionTreeClassifier(random_state=42)

# Allenamento e predizione
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)

# Calcolo della matrice di confusione
CM += confusion_matrix(y_test, y_pred)
y_test_all.extend(y_test)
predictions_all.extend(y_pred)

print("Matrice di Confusione Cumulativa:")
print(CM)


Matrice di Confusione Cumulativa:
[[3. 6.]
 [7. 8.]]


In [235]:
#RANDOM FOREST

# Divisione train/test con proporzione 70%-30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Gestione valori mancanti
numerical_imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(numerical_imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(numerical_imputer.transform(X_test), columns=X_test.columns)

# Modello Random Forest
rf = RandomForestClassifier(random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

# Predizioni
y_pred = rf.predict(X_test)

# Matrice di Confusione e Report
CM = confusion_matrix(y_test, y_pred)
print("Matrice di Confusione:")
print(CM)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Matrice di Confusione:
[[ 2  7]
 [ 4 11]]
Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.22      0.27         9
           1       0.61      0.73      0.67        15

    accuracy                           0.54        24
   macro avg       0.47      0.48      0.47        24
weighted avg       0.51      0.54      0.52        24



In [239]:
#SVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# Divisione train/test con proporzione 70%-30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Gestione valori mancanti
numerical_imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(numerical_imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(numerical_imputer.transform(X_test), columns=X_test.columns)


# Creazione della pipeline per preprocessing e classificatore
svc_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scaling dei dati
    ('svc', SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42))  # Modello SVC
])

# Allenamento
svc_pipeline.fit(X_train, y_train)

# Predizioni
y_pred = svc_pipeline.predict(X_test)

# Matrice di confusione e report
CM = confusion_matrix(y_test, y_pred)
print("Matrice di Confusione:")
print(CM)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Matrice di Confusione:
[[ 2  7]
 [ 2 13]]

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.22      0.31         9
           1       0.65      0.87      0.74        15

    accuracy                           0.62        24
   macro avg       0.57      0.54      0.53        24
weighted avg       0.59      0.62      0.58        24



In [245]:
from sklearn.model_selection import GridSearchCV

# Definizione della griglia di iperparametri
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [1, 0.1, 0.01, 0.001, 'scale'],
    'svc__kernel': ['rbf', 'linear', 'poly']
}

# GridSearch con pipeline
grid_search = GridSearchCV(svc_pipeline, param_grid, cv=5, scoring='accuracy', verbose=2)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

# Miglior modello
best_svc = grid_search.best_estimator_
y_pred = best_svc.predict(X_test)

# Matrice di Confusione e Report
CM = confusion_matrix(y_test, y_pred)
print("Matrice di Confusione:")
print(CM)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END ..........svc__C=0.1, svc__gamma=1, svc__kernel=rbf; total time=   0.0s
[CV] END ..........svc__C=0.1, svc__gamma=1, svc__kernel=rbf; total time=   0.0s
[CV] END ..........svc__C=0.1, svc__gamma=1, svc__kernel=rbf; total time=   0.0s
[CV] END ..........svc__C=0.1, svc__gamma=1, svc__kernel=rbf; total time=   0.0s
[CV] END ..........svc__C=0.1, svc__gamma=1, svc__kernel=rbf; total time=   0.0s
[CV] END .......svc__C=0.1, svc__gamma=1, svc__kernel=linear; total time=   0.0s
[CV] END .......svc__C=0.1, svc__gamma=1, svc__kernel=linear; total time=   0.0s
[CV] END .......svc__C=0.1, svc__gamma=1, svc__kernel=linear; total time=   0.0s
[CV] END .......svc__C=0.1, svc__gamma=1, svc__kernel=linear; total time=   0.0s
[CV] END .......svc__C=0.1, svc__gamma=1, svc__kernel=linear; total time=   0.0s
[CV] END .........svc__C=0.1, svc__gamma=1, svc__kernel=poly; total time=   0.0s
[CV] END .........svc__C=0.1, svc__gamma=1, svc

In [246]:
print(classification_report(y_test,
                            y_pred))

              precision    recall  f1-score   support

           0       0.60      0.67      0.63         9
           1       0.79      0.73      0.76        15

    accuracy                           0.71        24
   macro avg       0.69      0.70      0.70        24
weighted avg       0.72      0.71      0.71        24



In [247]:
print("Distribuzione delle classi nel training set:")
print(y_train.value_counts())
print("\nDistribuzione delle classi nel test set:")
print(y_test.value_counts())


Distribuzione delle classi nel training set:
Recidiva/Non_Recidiva
1    30
0    25
Name: count, dtype: int64

Distribuzione delle classi nel test set:
Recidiva/Non_Recidiva
1    15
0     9
Name: count, dtype: int64


In [248]:
# Valutazione globale del modello
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nMetriche globali sul Test Set:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


Metriche globali sul Test Set:
Accuracy: 0.71
Precision: 0.79
Recall: 0.73
F1-Score: 0.76


In [251]:
#DECISION TREE
# Funzione per testare tutti i dati nel test set
def test_full_test_set():
    # Trasforma il test set con il selettore di feature
    #X_test_selected = kf.transform(X_test)

    # Predizioni sul test set
    predictions = dtree.predict(X_test)

    # Confronto con i valori reali
    correct_predictions = (y_test == predictions).sum()
    total_predictions = len(y_test)
    accuracy_on_test = correct_predictions / total_predictions

    print("\nRisultati sui dati di Test Set:")
    for i, pred in enumerate(predictions):
        print(f"Esempio {i+1}: {'Recidiva' if pred == 1 else 'Non Recidiva'} (Reale: {'Recidiva' if y_test.iloc[i] == 1 else 'Non Recidiva'})")

    print(f"\nNumero di predizioni corrette: {correct_predictions}")
    print(f"Totale predizioni: {total_predictions}")
    print(f"Accuracy sul Test Set: {accuracy_on_test:.2f}")


In [ ]:
#SVC
# Funzione per testare tutti i dati nel test set
def test_full_test_set():
    """
    Testa il modello SVC sui dati del test set e calcola l'accuratezza.
    
    Args:
        svc_pipeline: La pipeline SVC già allenata.
        X_test: I dati di test.
        y_test: I valori reali del test set.
    """
    # Predizioni sul test set
    predictions = best_svc.predict(X_test)

    # Calcolo dell'accuratezza
    correct_predictions = (y_test == predictions).sum()
    total_predictions = len(y_test)
    accuracy_on_test = correct_predictions / total_predictions

    print("\nRisultati sui dati del Test Set:")
    for i, pred in enumerate(predictions):
        print(f"Esempio {i+1}: {'Recidiva' if pred == 1 else 'Non Recidiva'} "
              f"(Reale: {'Recidiva' if y_test.iloc[i] == 1 else 'Non Recidiva'})")

    print(f"\nNumero di predizioni corrette: {correct_predictions}")
    print(f"Totale predizioni: {total_predictions}")
    print(f"Accuracy sul Test Set: {accuracy_on_test:.2f}")

In [252]:
test_full_test_set()


Risultati sui dati del Test Set:
Esempio 1: Recidiva (Reale: Recidiva)
Esempio 2: Non Recidiva (Reale: Non Recidiva)
Esempio 3: Recidiva (Reale: Recidiva)
Esempio 4: Non Recidiva (Reale: Recidiva)
Esempio 5: Recidiva (Reale: Recidiva)
Esempio 6: Non Recidiva (Reale: Recidiva)
Esempio 7: Non Recidiva (Reale: Non Recidiva)
Esempio 8: Recidiva (Reale: Recidiva)
Esempio 9: Recidiva (Reale: Non Recidiva)
Esempio 10: Recidiva (Reale: Recidiva)
Esempio 11: Recidiva (Reale: Non Recidiva)
Esempio 12: Non Recidiva (Reale: Non Recidiva)
Esempio 13: Recidiva (Reale: Recidiva)
Esempio 14: Recidiva (Reale: Recidiva)
Esempio 15: Non Recidiva (Reale: Recidiva)
Esempio 16: Recidiva (Reale: Recidiva)
Esempio 17: Non Recidiva (Reale: Non Recidiva)
Esempio 18: Non Recidiva (Reale: Non Recidiva)
Esempio 19: Non Recidiva (Reale: Non Recidiva)
Esempio 20: Non Recidiva (Reale: Recidiva)
Esempio 21: Recidiva (Reale: Recidiva)
Esempio 22: Recidiva (Reale: Non Recidiva)
Esempio 23: Recidiva (Reale: Recidiva)
Es